In [1]:

# Importing necessary libraries
import torch
import numpy as np
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer



# Define the model name and torch device
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the tokenizer and model
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df = pd.read_csv("llm_strycture.csv")
df.head()


,Setting and Description
0,Optimization Settings: Generate code to add a ...
1,Enhanced GPU Utilization: Write a function tha...
2,Data Augmentation: Provide code to implement d...
3,Error Handling in Dataset Loading: Create erro...
4,Quantum Circuit Parameter Tuning: Suggest code...


In [5]:
#i want to iterate all the elements of df to list
df_list = df["Setting and Description"].values.tolist()
df_list


['Optimization Settings: Generate code to add a learning rate scheduler to the optimizer in the Quantum Hybrid Neural Network to improve training efficiency.',
 'Enhanced GPU Utilization: Write a function that ensures the HybridNet model and all tensors are moved to GPU if available, optimizing memory and computation speed.',
 'Data Augmentation: Provide code to implement data augmentation in the DatasetHandler class for the training dataset to enhance model generalizability.',
 'Error Handling in Dataset Loading: Create error handling in the DatasetHandler class for scenarios where data paths or labels cannot be correctly loaded or parsed.',
 'Quantum Circuit Parameter Tuning: Suggest code modifications to allow dynamic adjustment of quantum circuit parameters such as the number of qubits and layers based on dataset features.',
 'Real-time Training Metrics: Generate code for real-time visualization of training loss and validation accuracy during each epoch using matplotlib or another 

https://github.com/adarshgowdaa/pegasus-paraphrase/blob/main/main.py

In [6]:
def get_response(input_text, num_return_sequences):
    batch = tokenizer.prepare_seq2seq_batch(
        [input_text], truncation=True, padding='longest', max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch, max_length=60, num_beams=10,
                                num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [7]:
#creaye a 2d array of size df list with 10 columns
response = np.empty((len(df_list), 10), dtype=object)

idx = 0
for i in df_list:
    context = i
    num_return_sequences = 10
    num_beams = 10
    response[idx]=get_response(context, num_beams)
    idx = idx + 1

#get_response(context, num_return_sequences)
#context = "Initialize quantum registers to represent data points and cluster centroids."

#val = get_response(context, num_return_sequences)

response

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transfor

array([['Generate code to add a learning rate scheduler to the optimizer to improve training efficiency.',
        'Generate code to add a learning rate scheduler to the optimizer to improve training efficiency',
        'Generate code to add a learning rate scheduler to the optimizer in the quantum hybrid neural network to improve training efficiency.',
        'Adding a learning rate scheduler to the optimizer will improve training efficiency.',
        'Generate code to add a learning rate scheduler to the optimizer in the quantum hybrid neural network to improve training efficiency',
        'Generate code to add a learning rate scheduler to the optimizer in the quantum hybrid neural network.',
        'To improve training efficiency, add a learning rate scheduler to the optimizer.',
        'Generate code to add a learning rate scheduler to the optimizer in the quantum hybrid Neural Network to improve training efficiency.',
        'Adding a learning rate scheduler to the optimize

In [8]:
#convert response to csv file
df_response = pd.DataFrame(response)
df_response.to_csv('generated_prompts.csv', index=False)


In [9]:
#lavenshtein distance between two strings   
def lev(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([lev(s[:-1], t)+1,
               lev(s, t[:-1])+1, 
               lev(s[:-1], t[:-1]) + cost])
            
    

    
    return res



In [10]:

def similarity_score(s1, s2):
    value = editdistance.eval(s1, s2)
    return 1 - value / max(len(s1), len(s2))

In [11]:
similarity_score(context,val[0])
len(val)
   

NameError: name 'val' is not defined

In [12]:
sim = []

for i in range(len(val)):
    #create an aaray to store the index and similarity score
  
    sim.append(similarity_score(context,val[i]))
print(sim)
#convert sim to 2d array with index and similarity score and add index

sim = np.array(sim)
sim = sim.reshape(-1,1)
sim = np.hstack((sim, np.arange(len(sim)).reshape(-1,1)))
sim = sim[sim[:,0].argsort()[::-1]]
sim

NameError: name 'val' is not defined

In [ ]:
#Select the similarity score greater than 0.5
sim = sim[sim[:,0]>0.9]
sim

array([], shape=(0, 2), dtype=float64)